In [27]:
import pandas as pd

In [28]:
df = pd.read_csv("data/gemstone.csv")

In [29]:
df.drop(columns="id", axis= 1, inplace=True)

In [30]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [31]:
X = df.drop(columns='price', axis=1)
y = df[['price']]

In [32]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [33]:
numerical_cols = X.select_dtypes(exclude=object).columns
categorical_cols = X.select_dtypes(include=object).columns

In [34]:
numerical_cols
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [35]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [36]:
from sklearn.impute import SimpleImputer #Handling Missing Values
from sklearn.preprocessing import StandardScaler #Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
from sklearn.pipeline import Pipeline #Making Pipelines
from sklearn.compose import ColumnTransformer

In [37]:
# Building Numerical Pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Building Categorical Pipeline
cat_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('Ordinalencoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories]))    
        ('Scaler', StandardScaler())
        
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])

In [38]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('Scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=30)

In [41]:
X_train

,carat,cut,color,clarity,depth,table,x,y,z
168192,0.34,Ideal,I,VVS2,60.9,57.0,4.56,4.53,2.76
35202,0.90,Good,E,SI1,63.8,57.0,6.07,6.03,3.87
41091,1.02,Premium,G,VS1,62.7,58.0,6.35,6.39,4.00
31239,0.32,Premium,G,VS2,62.1,59.0,4.37,4.35,2.71
45722,0.35,Ideal,J,VVS2,61.1,56.0,4.53,4.57,2.78
...,...,...,...,...,...,...,...,...,...
66455,0.31,Ideal,E,SI1,61.8,56.0,4.31,4.35,2.68
46220,1.25,Ideal,G,SI2,62.0,56.0,6.88,6.95,4.28
98804,1.00,Good,G,SI1,63.5,56.0,6.29,6.37,4.02
48045,1.10,Ideal,G,VS1,59.9,60.0,6.68,6.77,4.01


In [42]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [43]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [44]:

print(X_train)

        num_pipeline__carat  num_pipeline__depth  num_pipeline__table  \
0                 -0.975439            -0.849607            -0.121531   
1                  0.235195             1.833637            -0.121531   
2                  0.494617             0.815855             0.399800   
3                 -1.018676             0.260701             0.921131   
4                 -0.953821            -0.664555            -0.642862   
...                     ...                  ...                  ...   
135496            -1.040295            -0.016876            -0.642862   
135497             0.991842             0.168176            -0.642862   
135498             0.451380             1.556060            -0.642862   
135499             0.667565            -1.774863             1.442462   
135500             0.256814             0.815855            -0.121531   

        num_pipeline__x  num_pipeline__y  num_pipeline__z  cat_pipeline__cut  \
0             -1.042757        -1.080970   

In [45]:
print(numerical_cols)
print("*************************")
print(categorical_cols)
print("*************************")

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')
*************************
Index(['cut', 'color', 'clarity'], dtype='object')
*************************


In [46]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [47]:
regression = LinearRegression()
regression.fit(X_train,y_train)
regression.predict(X_test)

array([[ 1944.61745133],
       [ 1877.82779382],
       [ 1475.99304703],
       ...,
       [ 1863.94967422],
       [  173.6941081 ],
       [14971.98464342]])

In [48]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [49]:
regression.intercept_

array([3970.76628955])

In [50]:
import numpy as np
def model_evaluation(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    r2score = r2_score(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    return mae, mse, r2score, rmse

In [51]:
models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "ElasticNet" : ElasticNet()
}

model_list= []
trained_model_list=[]
r2_list=[]
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_pred= model.predict(X_test)
    mae, mse, r2score, rmse = model_evaluation(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Training Performance")
    print("rmse: ", rmse)
    print("mae: ", mae)
    print("mse: ", mse)
    print("r2_score: ", r2score)
    print("\n*********************************\n")

    


LinearRegression
Model Training Performance
rmse:  1013.9047094344004
mae:  674.0255115796832
mse:  1028002.7598132559
r2_score:  0.9368908248567511

*********************************

Lasso
Model Training Performance
rmse:  1013.8784226767013
mae:  675.0716923362161
mse:  1027949.4559693959
r2_score:  0.9368940971841704

*********************************

Ridge
Model Training Performance
rmse:  1013.9059272771647
mae:  674.0555800798206
mse:  1028005.2293677672
r2_score:  0.9368906732505937

*********************************

ElasticNet
Model Training Performance
rmse:  1533.4162456064048
mae:  1060.7368759154729
mse:  2351365.382289642
r2_score:  0.8556494831165181

*********************************



In [52]:
list(models.keys())[i]


'ElasticNet'

In [65]:
a = pd.read_csv("C:\\Projects\\DiamondPricePrediction\\artifacts\\train.csv")

In [66]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135501 entries, 0 to 135500
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       135501 non-null  int64  
 1   carat    135501 non-null  float64
 2   cut      135501 non-null  object 
 3   color    135501 non-null  object 
 4   clarity  135501 non-null  object 
 5   depth    135501 non-null  float64
 6   table    135501 non-null  float64
 7   x        135501 non-null  float64
 8   y        135501 non-null  float64
 9   z        135501 non-null  float64
 10  price    135501 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 11.4+ MB


In [68]:
X_train.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135501 entries, 0 to 135500
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   num_pipeline__carat    135501 non-null  float64
 1   num_pipeline__depth    135501 non-null  float64
 2   num_pipeline__table    135501 non-null  float64
 3   num_pipeline__x        135501 non-null  float64
 4   num_pipeline__y        135501 non-null  float64
 5   num_pipeline__z        135501 non-null  float64
 6   cat_pipeline__cut      135501 non-null  float64
 7   cat_pipeline__color    135501 non-null  float64
 8   cat_pipeline__clarity  135501 non-null  float64
dtypes: float64(9)
memory usage: 9.3 MB
